Data about unemployment on municipality level are loaded from https://www.regionalstatistik.de/genesis//online?operation=table&code=13211-01-03-5&bypass=true&levelindex=1&levelid=1762338172400#abreadcrumb

Data contain ID column as identifier.

ID consist of:
- Land (2 digits) which represents Bundesland
- Regierungsbezirk (1 digit) which represents governmental district
- Kreis (2 digits) which represents district
- Gemeinde (3 digits) which represents municipality

There are also rows representing higher administrative levels (e.g. districts) which can be identified by having just two or five digits in ID column.
We cannot just filter them out because Hamburg has no districts and its municipalities have just two digits in ID column.

# Load of unemployment data

In [ ]:
import pandas as pd

types = {
    "unemployment_total": "float",
    "ausländisch": "float",
    "schwerbehindert": "float",
    "15 bis unter 20 Jahre": "float",
    "15 bis unter 25 Jahre": "float",
    "55 bis unter 65 Jahre": "float",
    "langzeitarbeitslos": "float",
}
df = pd.read_csv(
    "../data/raw/unemployment.csv",
    sep=";",
    encoding="latin1",
    skiprows=7,
    skipfooter=4,
    na_values=["-", "."],
)
# rename columns
df.rename(columns={"Unnamed: 0": "ID", "Unnamed: 1": "Municipality", "Unnamed: 2": "unemployment_total"}, inplace=True)
# drop first row which contains whole germany data
df = df.iloc[1:]

# change dtypes by dictionary
for col, dtype in types.items():
    df[col] = df[col].astype(dtype)


# enrich data with land, kreis, gemeinde
df["Land"] = df["ID"].str[:2]
df["Kreis"] = df["ID"].str[2:5]
df["Gemeinde"] = df["ID"].str[5:]

df

In [ ]:
df.dtypes

In [ ]:
df["ausländisch"].astype("float")

In [ ]:
df[df["ID"] == "01"]

In [ ]:
df[(df["ID"].str.len() == 5) & (df["ID"].str.startswith("01"))].groupby("Land").sum()

In [ ]:
# add people count and calculate unemployment rate
from geoscore_de.data_flow.population import load_population_data

people_count = load_population_data("../data/raw/population.csv")
people_count[people_count["age_group"] == "Insgesamt"]

In [ ]:
df_merged = df.merge(
    people_count[people_count["age_group"] == "Insgesamt"][["AGS", "people_count"]],
    left_on="ID",
    right_on="AGS",
)

In [ ]:
df_merged["unemployment_rate"] = df_merged["unemployment_total"] / df_merged["people_count"]

In [ ]:
df_merged["unemployment_rate"]

In [ ]:
import plotnine as gg

# show histogram of unemployment rate
(
    gg.ggplot(df_merged, gg.aes(x="unemployment_rate"))
    + gg.geom_histogram(binwidth=0.001)
    + gg.scale_x_continuous(labels=lambda labels: ["{:.0f}%".format(x * 100) for x in labels])
    + gg.labs(
        title="Distribution of Unemployment Rate in German Municipalities",
        x="Unemployment Rate",
        y="Count of Municipalities",
    )
)

In [ ]:
# now show it on map of germany
import geopandas as gpd

gdf = gpd.read_file("../data/gemeinden_simplify200.geojson")
gdf["ID"] = gdf["AGS"].astype("string")

In [ ]:
# join data by ags
gdf_merged = gdf.merge(df_merged, on="ID", how="left", indicator=True)

In [ ]:
# print count of missing values
gdf_merged["_merge"].value_counts()

In [ ]:
import matplotlib.pyplot as plt

gdf_merged.plot(
    column="unemployment_rate",
    legend=True,
    cmap="coolwarm",
    figsize=(10, 10),
    vmin=0,
    vmax=0.04,
    missing_kwds={"color": "black"},
)
plt.title("Unemployment Rate in German Municipalities")
plt.show()